In [1]:
import duckdb
import pandas as pd

In [2]:
SOURCE = '../data/kilt_wiki_small.duckdb'
WIKI = '../data/kilt_wiki_small.jsonl'
PARAGRAPHS = '../data/kilt_wiki_small_paragraphs.jsonl'

In [3]:
df = pd.read_json(WIKI, orient='records', lines=True)

df_wiki = df[['kilt_id', 'wikipedia_id', 'wikipedia_title', 'text', 'categories']]
df_wiki.head(1)


,kilt_id,wikipedia_id,wikipedia_title,text,categories
0,339,339,Ayn Rand,"{'paragraph': ['Ayn Rand ', 'Ayn Rand (; born ...","American women novelists,American abortion-rig..."


In [4]:
df_p = pd.read_json(PARAGRAPHS, orient='records', lines=True)

df_p_small = df_p[['kilt_id', 'paragraph_id', 'paragraph']]
df_p_small.head(1)

,kilt_id,paragraph_id,paragraph
0,76383,0,The Ten Commandments (1956 film)\n


In [5]:
con = duckdb.connect(SOURCE)

In [6]:
con.execute("SELECT count(*) FROM paragraph;").fetchall()[0][0]

146644

In [10]:
con.sql("DROP TABLE IF EXISTS wiki")
con.sql('CREATE TABLE wiki AS SELECT * FROM df_wiki')
con.sql("INSERT INTO wiki SELECT * FROM df_wiki")

con.sql("DROP TABLE IF EXISTS paragraph")
con.sql("CREATE TABLE paragraph (wikipedia_id VARCHAR, wikipedia_title VARCHAR, id BIGINT, index INTEGER, text VARCHAR);")
con.sql("CREATE SEQUENCE paragraph_id START 1;")

con.execute("""
INSERT INTO paragraph
SELECT
    wikipedia_id,
    wikipedia_title,
    nextval('paragraph_id') as id,
    idx AS index,
    paragraph AS text
FROM wiki, UNNEST(text.paragraph) WITH ORDINALITY AS t(paragraph, idx);
""")

# con.sql('CREATE TABLE paragraph AS SELECT * FROM df_p_small')
# con.sql("INSERT INTO paragraph SELECT * FROM df_p_small")

In [11]:
con.execute("SHOW ALL TABLES").df()

,database,schema,name,column_names,column_types,temporary
0,kilt_wiki_small,main,paragraph,"[wikipedia_id, wikipedia_title, id, index, text]","[VARCHAR, VARCHAR, BIGINT, INTEGER, VARCHAR]",False
1,kilt_wiki_small,main,wiki,"[kilt_id, wikipedia_id, wikipedia_title, text,...","[BIGINT, BIGINT, VARCHAR, STRUCT(paragraph VAR...",False


In [12]:
con.execute("SELECT * FROM wiki LIMIT 5").df()

,kilt_id,wikipedia_id,wikipedia_title,text,categories
0,339,339,Ayn Rand,"{'paragraph': ['Ayn Rand ', 'Ayn Rand (; born ...","American women novelists,American abortion-rig..."
1,358,358,Algeria,"{'paragraph': ['Algeria ', 'Algeria ( ; , ; ),...","Arab republics,Algeria,States and territories ..."
2,20134,20134,Minnesota Vikings,"{'paragraph': ['Minnesota Vikings ', 'The Minn...","Minnesota Vikings,American football teams esta..."
3,420438,420438,Steven Tyler,"{'paragraph': ['Steven Tyler ', 'Steven Tyler ...","1948 births,Songwriters from Massachusetts,20t..."
4,269859,269859,A Perfect Circle,"{'paragraph': ['A Perfect Circle ', 'A Perfect...","Alternative metal supergroups,Musical quintets..."


In [13]:
con.execute("SELECT * FROM paragraph LIMIT 5").df()

,wikipedia_id,wikipedia_title,id,index,text
0,339,Ayn Rand,1,101,"BULLET::::- ""Writings of Ayn Rand"" – from C-SP..."
1,358,Algeria,2,197,BULLET::::- EU Neighbourhood Info Centre: Alge...
2,20134,Minnesota Vikings,3,262,BULLET::::- List of Minnesota Vikings broadcas...
3,420438,Steven Tyler,4,81,"BULLET::::- 2011 – ""Outstanding Performer In A..."
4,269859,A Perfect Circle,5,54,"BULLET::::- ""Eat the Elephant"" (2018)\n"


In [ ]:
result = con.execute("SELECT * FROM paragraph LIMIT 5 OFFSET 5").df()
result

,wikipedia_id,wikipedia_title,id,index,text
0,595,Andre Agassi,6,137,BULLET::::- Agassi's Tennis Hall of Fame Induc...
1,206724,Anne Bancroft,7,39,BULLET::::- Anne Bancroft at the University of...
2,206705,Colin Farrell,8,60,"BULLET::::- ""New Yorker"" caricatures of Farrel..."
3,23861381,Bradford,9,207,BULLET::::- Sunrise and sunset times\n
4,21909903,Catching Fire,10,26,BULLET::::- Scholastic official website\n


: 

In [6]:
BATCH_ROWS = 512
cursor = con.execute("SELECT * FROM paragraph")

rows = cursor.fetchmany(BATCH_ROWS)
rows

[('339',
  'Ayn Rand',
  101,
  'BULLET::::- "Writings of Ayn Rand" – from C-SPAN\'s ""\n'),
 ('358',
  'Algeria',
  197,
  'BULLET::::- EU Neighbourhood Info Centre: Algeria\n'),
 ('20134',
  'Minnesota Vikings',
  262,
  'BULLET::::- List of Minnesota Vikings broadcasters\n'),
 ('420438',
  'Steven Tyler',
  81,
  'BULLET::::- 2011 – "Outstanding Performer In An Animated Program" – For playing "The Mad Hatter" on The Wonder Pets: Adventures in Wonderland (nominated)\n'),
 ('269859', 'A Perfect Circle', 54, 'BULLET::::- "Eat the Elephant" (2018)\n'),
 ('595',
  'Andre Agassi',
  137,
  "BULLET::::- Agassi's Tennis Hall of Fame Induction for Steffi Graf\n"),
 ('206724',
  'Anne Bancroft',
  39,
  "BULLET::::- Anne Bancroft at the University of Wisconsin's Actors Studio audio collection\n"),
 ('206705',
  'Colin Farrell',
  60,
  'BULLET::::- "New Yorker" caricatures of Farrell and Brendan Gleeson in "In Bruges"\n'),
 ('23861381', 'Bradford', 207, 'BULLET::::- Sunrise and sunset times\n

In [76]:
def get_paragraph_batch(index, batch_size = 25):
    return con.execute("SELECT * FROM paragraph LIMIT ? OFFSET ?", [batch_size, index*batch_size]).df()


get_paragraph_batch(1)

,wikipedia_id,wikipedia_title,paragraph_id,text
0,34604303,Quartet (2012 film),49,"Lisa Schwarzbaum of ""Entertainment Weekly"" gav..."
1,2025815,Louise Simonson,105,BULLET::::- Louise Simonson at the Unofficial ...
2,23882849,Taylor Schilling,13,BULLET::::- LGBTQ culture in New York City\n
3,1218644,Jordan Knight,39,BULLET::::- Trans Continental Records\n
4,158889,George Clooney,69,"Throughout his career, Clooney has won two Aca..."
5,209066,Reba McEntire,117,BULLET::::- Reba McEntire Profile on Country M...
6,22941812,Furia (film),11,BULLET::::- Étienne Chicot as Quicailler\n
7,75912,Time Lord,102,BULLET::::- Time War (Doctor Who)\n
8,76033,The Great Gatsby,105,"BULLET::::- An Index to ""The Great Gatsby""\n"
9,52281037,Inhumans (TV series),89,BULLET::::- General references\n
